# Intérroger des documents (RAG)

Commencez par télécharger le document qui va vous servir de base de connaissance.

In [1]:
# Download a file
import wget
import os

file_path = "my_document.pdf"
if os.path.exists(file_path):
    os.remove(file_path)
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

'my_document.pdf'

Puis instancier la connexion à l'API Albert.

In [19]:
# OpenAI client configuration
import requests
from openai import OpenAI

base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = os.getenv("API_KEY")

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_. 

In [25]:
# Remove previous files

collection = "leo"
#response = session.delete(f"{base_url}/files/{collection}")
#response.status_code

In [4]:
# Upload a file
model = "intfloat/multilingual-e5-large"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': '65773377-86ce-4617-a678-b9a03e8d6575',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files.`

In [5]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")
response.json()

{'object': 'list',
 'data': [{'object': 'file',
   'id': '65773377-86ce-4617-a678-b9a03e8d6575',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'chunk_ids': ['0b0a35ff-8dad-45c1-bab9-bf0c34bd78fd',
    '25737bba-3c18-42f9-ac19-5f8e471d5fd5',
    '2c0c317c-524d-42dd-8ca4-05bda9428224',
    '34b7c8d8-18d8-45b2-b9b6-1a65e13ab6d6',
    '36298208-0864-492c-9ee0-b87d15b96b34',
    '4b3992d7-d014-411a-bb24-7af5c0edeff5',
    '5a4f51b5-6726-4a36-9cf4-9df8f861d89c',
    '707356ed-67de-4427-9be9-52f0b57c6c78',
    '795074c3-09da-4882-93e4-8ce157370530',
    '85e69ae6-6c7c-4fc9-8ffe-39f61af49c7f',
    '9432adec-a7a8-4c44-92df-331b1b3f8c11',
    '9fe970a4-9062-4059-9fde-4628165caf52',
    'a380c886-998e-4f20-afa0-59cbc1fd1012',
    'cd145dc3-90fb-4834-9da5-b2536d1611b7',
    'd07c2ad0-b57d-4a60-b232-37142eaf2eed',
    'd835f64d-8f78-4978-bc03-104432865792'],
   'created_at': 1722794075}]}

In [6]:
file_id = response.json()["data"][0]["id"]

In [7]:
# Display tools parameters
response = session.get(f"{base_url}/tools")
for tool in response.json()["data"]:
    if tool["id"] == "BaseRAG":
        print(tool["description"].strip())

Base RAG, basic retrival augmented generation.

    Args:
        embeddings_model (str): OpenAI embeddings model
        collection (Optional[List[str]], optional): List of collections to search in. Defaults to None (all collections).
        file_ids (Optional[List[str]], optional): List of file IDs in the selected collections (after upload files). Defaults to None (all files are selected).
        k (int, optional): Top K per collection (max: 6). Defaults to 4.
        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.

    DEFAULT_PROMPT_TEMPLATE:
        "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}

Documents :

{files}"


In [26]:
# Chat completions
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Qui est Ulrich Tan ?"}],
    "stream": False,
    "n": 1,
    "tools": [
        {
            "function": {
                "name": "BaseRAG",
                "parameters": {
                    "embeddings_model": model,
                    "collections": [collection],
                    "k": 2,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Selon les documents, il n'y a pas d'Ulrich Tan mentionné. Les noms et les fonctionnaires énumérés ne comprennent pas Ulrich Tan.


In [22]:
print(response.metadata[0]["BaseRAG"]["prompt"])

[{'BaseRAG': {'prompt': 'Réponds à la question suivante en te basant sur les documents ci-dessous : Qui est Ulrich Tan ?\n\nDocuments :\n\n–  Mme Elsa Le Duigou, agent contractuel ; –  M. Valentin Brel, agent contractuel ; –  Mme Margot Sanchez, agent contractuel ; –  M. Paul Burgun, agent contractuel ; –  Mme Emma Ghariani, agent contractuel. Art.  5.  –  Délégation est donnée à Mme Géraldine Taurand, attachée d’administration hors classe, cheffe de la mission « Budget et achats », pour signer tous actes de gestion et procéder aux validations électronique dans l’outil',
   'metadata': {'chunks': [{'file_id': '0fb8a999-6e4e-40f4-bcd8-807541687e94',
      '_id': '7b19aa26-7f0e-4a19-9441-cffa848c0e8c',
      '_collection_name': 'k7se0ojelcrinzkk-leo'}]}}}]